In [17]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_lg")
doc = nlp("Alex Smith worked at Acme Corp Inc.")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Alex Smith', 'PERSON'), ('Acme Corp Inc.', 'ORG')]


In [19]:
displacy.render(doc, jupyter=True, style="dep")

In [16]:
person_entities = [ent for ent in doc.ents if ent.label_ == "PERSON"]
for ent in person_entities:
    head = ent.root.head
    print([(ent.text, ent.label_, head)])
    if head.lemma_ == "work":
        preps = [token for token in head.children if token.dep_ == "prep"]
        for prep in preps:
            orgs = [token for token in prep.children if token.ent_type_ == "ORG"]
            print({"person": ent, "orgs": orgs, "past": head.tag_ == "VBD"})

[('Alex Smith', 'PERSON', worked)]
['_', '__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', 'ancestors', 'check_flag', 'children', 'cluster', 'conjuncts', 'dep', 'dep_', 'doc', 'ent_id', 'ent_id_', 'ent_iob', 'ent_iob_', 'ent_kb_id', 'ent_kb_id_', 'ent_type', 'ent_type_', 'get_extension', 'has_dep', 'has_extension', 'has_head', 'has_morph', 'has_vector', 'head', 'i', 'idx', 'iob_strings', 'is_alpha', 'is_ancestor', 'is_ascii', 'is_bracket', 'is_currency', 'is_digit', 'is_left_punct', 'is_lower', 'is_oov', 'is_punct', 'is_quote', 'is_right_punct', 'is_sent_end', 'is_sent_start', 'is_space', 'is_stop', 'is_title', 'is_upper', 'lang', 'lang_', 'left_edge', 'lefts', 'lemma', 'lemma_', '

In [167]:
def compounds_finder(doc):

    com_range = []
    max_ind = len(doc)
    for idx, tok in enumerate(doc):
        if((tok.dep_ == "compound") and (idx < max_ind)):
            com_range.append([idx, idx+1])

    to_remove = []
    intersections = []
    for t1 in com_range:
        for t2 in com_range:
            if(t1 != t2):
                s1 = set(t1)
                s2 = set(t2)
                if(len(s1.intersection(s2)) > 0):
                    to_remove.append(t1)
                    to_remove.append(t2)
                    union = list(s1.union(s2))
                    if union not in intersections:
                        intersections.append(union)

    result = [t for t in com_range if t not in to_remove]
    print(result)
    print(intersections)
    return result + intersections

In [171]:
from spacy.tokens import Token, Span

class Compounds_Chain(object):
    name = 'compounds_chain' # the name of the component in the pipeline
    
    def __init__(self, nlp, label="CompChain"):
        
        # get label ID from the nlp vocabulary
        self.label = nlp.vocab.strings[label]
        
        # register the extensions
        Token.set_extension("is_compound_chain", default=False)
    
    def find_compouds(doc):
        com_range = []
        max_ind = len(doc)
        for idx, tok in enumerate(doc):
            if((tok.dep_ == "compound") and (idx < max_ind)):
                com_range.append([idx, idx+1])

        to_remove = []
        intersections = []
        for t1 in com_range:
            for t2 in com_range:
                if(t1 != t2):
                    s1 = set(t1)
                    s2 = set(t2)
                    if(len(s1.intersection(s2)) > 0):
                        to_remove.append(t1)
                        to_remove.append(t2)
                        union = list(s1.union(s2))
                        if union not in intersections:
                            intersections.append(union)

        result = [t for t in com_range if t not in to_remove]
    
        return result + intersections
    
    def __call__(self, doc):
        
        compound_ranges = find_compouds(doc)
        
        spans = [] 
        for cr in compound_ranges:
            entiry = 
    

In [170]:
doc1 = nlp("leadership experience in Product Management; demonstrated success launching and driving. Alex Smith worked at Acme Corp Inc.")
r = compounds_finder(doc1)
r

[[0, 1], [3, 4], [12, 13]]
[[16, 17, 18]]


[[0, 1], [3, 4], [12, 13], [16, 17, 18]]

In [169]:
print(compounds_finder(doc))

[[0, 1]]
[[4, 5, 6]]
[[0, 1], [4, 5, 6]]


In [76]:
for t in doc1:
    print(t.text + " " + t.dep_ + " " + t.pos_)

leadership compound NOUN
experience nsubj NOUN
in prep ADP
Product compound PROPN
Management pobj PROPN
; punct PUNCT
demonstrated ROOT VERB
success dobj NOUN
launching acl VERB
and cc CCONJ
driving conj NOUN
. punct PUNCT


In [58]:
doc[0].ancestors